In [11]:
import pandas as pd
import json
import os

all_data = []

directory = r'C:\Users\gsuma\Documents\Sumanth\DAB2\DAB\Sem 4\Capstone\ipl_json (1)'

for file_id in range(335982, 1370354):  # Adjust the range as necessary
    file_path = os.path.join(directory, f'{file_id}.json')
    
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        match_details = data.get('info', {})
        match_city = match_details.get('city', '')
        match_date = match_details.get('dates', [])[0] if match_details.get('dates', []) else ''
        match_teams = " vs ".join(match_details.get('teams', []))
        match_venue = match_details.get('venue', '')
        player_of_match = ", ".join(match_details.get('player_of_match', []))
        match_winner = match_details.get('outcome', {}).get('winner', 'No result')  # Extracting winner information
        teams = match_details.get('teams', [])


        for inning in data.get('innings', []):
            team_name = str(inning.get('team', '')).strip(),
            opposing_team = [team for team in teams if team != team_name][0]  # Identifying the opposing team

            for over in inning.get('overs', []):
                over_number = over.get('over', 0)
                for delivery in over.get('deliveries', []):
                    extras = delivery.get('extras', {})
                    extras_type = ", ".join(extras.keys())
                    extras_total = sum(extras.values()) if extras else 0
                    
                    # Correcting the extraction of wicket details
                    wicket_info = delivery.get('wicket', {})
                    wicket_type = wicket_info.get('kind', '')
                    player_out = wicket_info.get('player_out', '')
                    fielders = [fielder.get('name', '') for fielder in wicket_info.get('fielders', [])] if 'fielders' in wicket_info else []

                    delivery_data = {
                        'Match City': match_city,
                        'Match Date': match_date,
                        'Teams': match_teams,
                        'Venue': match_venue,
                        'Player of Match': player_of_match,
                        'Team': team_name,
                        'Over': over_number,
                        'Batsman': delivery.get('batter', ''),
                        'Runs': delivery.get('runs', {}).get('batter', 0),
                        'Extras Type': extras_type,
                        'Extras Total': extras_total,
                        'Wicket Type': wicket_type,
                        'Player Out': player_out,
                        'Bowler': delivery.get('bowler', ''),
                        'Winner': match_winner,  # Adding the winner field
                        'Opposing Team': opposing_team  # Adding the opposing team field

                    }
                    
                    if 'wickets' in delivery:
                        for wicket in delivery['wickets']:
                            delivery_data['Wicket Type'] = wicket.get('kind', '')
                            delivery_data['Player Out'] = wicket.get('player_out', '')
                    # Assuming multiple fielders could be involved (e.g., in the case of catches)
                            fielders = [fielder.get('name', '') for fielder in wicket.get('fielders', [])]
                            delivery_data['Fielder'] = ", ".join(fielders)
                    
                    # For simplicity, assuming one wicket per delivery; break after the first
                            break
                    all_data.append(delivery_data)

df = pd.DataFrame(all_data)



In [13]:
def remove_tuple_representation(df, column_name):
  
    # Convert the column to string type first
    df[column_name] = df[column_name].astype(str)
    
    # Now remove the tuple representation by stripping the characters '(', ')', and ','
    df[column_name] = df[column_name].str.replace(r"[\(',\)]", '', regex=True)
    
    return df

In [14]:
df = remove_tuple_representation(df, 'Team')


In [15]:
df

,Match City,Match Date,Teams,Venue,Player of Match,Team,Over,Batsman,Runs,Extras Type,Extras Total,Wicket Type,Player Out,Bowler,Winner,Opposing Team,Fielder
0,Bangalore,2008-04-18,Royal Challengers Bangalore vs Kolkata Knight ...,M Chinnaswamy Stadium,BB McCullum,Kolkata Knight Riders,0,SC Ganguly,0,legbyes,1,,,P Kumar,Kolkata Knight Riders,Royal Challengers Bangalore,NaN
1,Bangalore,2008-04-18,Royal Challengers Bangalore vs Kolkata Knight ...,M Chinnaswamy Stadium,BB McCullum,Kolkata Knight Riders,0,BB McCullum,0,,0,,,P Kumar,Kolkata Knight Riders,Royal Challengers Bangalore,NaN
2,Bangalore,2008-04-18,Royal Challengers Bangalore vs Kolkata Knight ...,M Chinnaswamy Stadium,BB McCullum,Kolkata Knight Riders,0,BB McCullum,0,wides,1,,,P Kumar,Kolkata Knight Riders,Royal Challengers Bangalore,NaN
3,Bangalore,2008-04-18,Royal Challengers Bangalore vs Kolkata Knight ...,M Chinnaswamy Stadium,BB McCullum,Kolkata Knight Riders,0,BB McCullum,0,,0,,,P Kumar,Kolkata Knight Riders,Royal Challengers Bangalore,NaN
4,Bangalore,2008-04-18,Royal Challengers Bangalore vs Kolkata Knight ...,M Chinnaswamy Stadium,BB McCullum,Kolkata Knight Riders,0,BB McCullum,0,,0,,,P Kumar,Kolkata Knight Riders,Royal Challengers Bangalore,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243812,Ahmedabad,2023-05-29,Gujarat Titans vs Chennai Super Kings,"Narendra Modi Stadium, Ahmedabad",DP Conway,Chennai Super Kings,14,S Dube,1,,0,,,MM Sharma,Chennai Super Kings,Gujarat Titans,NaN
243813,Ahmedabad,2023-05-29,Gujarat Titans vs Chennai Super Kings,"Narendra Modi Stadium, Ahmedabad",DP Conway,Chennai Super Kings,14,RA Jadeja,1,,0,,,MM Sharma,Chennai Super Kings,Gujarat Titans,NaN
243814,Ahmedabad,2023-05-29,Gujarat Titans vs Chennai Super Kings,"Narendra Modi Stadium, Ahmedabad",DP Conway,Chennai Super Kings,14,S Dube,1,,0,,,MM Sharma,Chennai Super Kings,Gujarat Titans,NaN
243815,Ahmedabad,2023-05-29,Gujarat Titans vs Chennai Super Kings,"Narendra Modi Stadium, Ahmedabad",DP Conway,Chennai Super Kings,14,RA Jadeja,6,,0,,,MM Sharma,Chennai Super Kings,Gujarat Titans,NaN


In [16]:
df.to_csv('IPL.csv', index=False)